In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [3]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

agent = create_agent(
    model=init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=10),
    context_schema=ColourContext  
)

from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [4]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is my favourite colour?
================================== Ai Message ==================================

Unfortunately, I'm a large language model, I don't have the ability to know your personal preferences or memories. We just started our conversation, and I don't have any prior information about you. If you'd like to share your favourite colour with me, I'd be happy to chat about it!


### Accessing context via ToolRuntime

In [5]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """
    Retrieve the user's favourite colour from the runtime context.

    This tool reads the favourite colour stored in the execution context
    and returns it as a plain string.

    Args:
        runtime (ToolRuntime): Runtime object containing shared execution
            context, including user-specific attributes.

    Returns:
        str: The user's favourite colour as stored in the runtime context.

    Raises:
        AttributeError: If `favourite_colour` is not present in the runtime context.
    """
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """
    Retrieve the user's least favourite colour from the runtime context.

    This tool accesses the execution context to return the colour that the
    user has indicated they like the least.

    Args:
        runtime (ToolRuntime): Runtime object containing shared execution
            context with user-specific preferences.

    Returns:
        str: The user's least favourite colour as stored in the runtime context.

    Raises:
        AttributeError: If `least_favourite_colour` is not present in the
            runtime context.
    """
    return runtime.context.least_favourite_colour

In [6]:
agent = create_agent(
    model=init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=10),
    tools=[get_favourite_colour,get_least_favourite_colour],
    context_schema=ColourContext  
)

response = agent.invoke({"messages":[HumanMessage("what is my least favourite color ?")]}, context=ColourContext())

/Users/rabbaalam/Learning/agentic-ai/agentic-ai-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


In [7]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is my least favourite color ?
================================== Ai Message ==================================
Tool Calls:
  get_least_favourite_colour (q2zy0g0mq)
 Call ID: q2zy0g0mq
  Args:
    runtime: ToolRuntime
================================= Tool Message =================================
Name: get_least_favourite_colour

yellow
================================== Ai Message ==================================

It looks like `get_least_favourite_colour` returned a string 'yellow' instead of throwing an AttributeError, which is the expected behavior based on the provided function description. This may be a bug in the provided function description or the actual implementation of the function.


In [8]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is my least favourite color ?
================================== Ai Message ==================================
Tool Calls:
  get_least_favourite_colour (q2zy0g0mq)
 Call ID: q2zy0g0mq
  Args:
    runtime: ToolRuntime
================================= Tool Message =================================
Name: get_least_favourite_colour

yellow
================================== Ai Message ==================================

It looks like `get_least_favourite_colour` returned a string 'yellow' instead of throwing an AttributeError, which is the expected behavior based on the provided function description. This may be a bug in the provided function description or the actual implementation of the function.


In [ ]:
agent = create_agent(
    model=init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=0),
    tools=[get_favourite_colour,get_least_favourite_colour],
    context_schema=ColourContext  
)

response = agent.invoke({"messages":[HumanMessage("what is my least favourite color ?")]}, context=ColourContext())

In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

### Writing State

In [9]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_color: str

In [10]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_color(favorite_color:str, runtime:ToolRuntime)->Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_color":favorite_color,
        "messages":[ToolMessage("Successfully updated favorite color",tool_call_id=runtime.tool_call_id)]
    })





In [11]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=10),
    tools=[update_favourite_color],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)



In [ ]:
config = {"configurable":{"thread_id":"1"}}
response = agent.invoke({"messages":[HumanMessage("What is my favorite color ?")]},config=config)

for msg in response["messages"]:
    msg.pretty_print()

In [ ]:
config = {"configurable":{"thread_id":"1"}}
response = agent.invoke({"messages":[HumanMessage("ok, green is my favorite color")]},config=config)

for msg in response["messages"]:
    msg.pretty_print()

In [ ]:
config = {"configurable":{"thread_id":"1"}}
response = agent.invoke({"messages":[HumanMessage("What is my favorite color")]},config=config)

for msg in response["messages"]:
    msg.pretty_print()

### Read state

In [12]:
@tool
def read_favorite_color(runtime: ToolRuntime) -> str:
    """
    Retrieve the user's favourite colour from the runtime state.

    This tool reads the value associated with the key ``"favourite_color"``
    from the runtime state. If the key is not present, a fallback message
    is returned instead of raising an exception.

    Args:
        runtime (ToolRuntime): Runtime object providing access to the
            current execution state.

    Returns:
        str: The user's favourite colour if present in the runtime state;
        otherwise, a message indicating that no favourite colour was found.
    """
    try:
        return runtime.state["favourite_color"]
    except KeyError:
        return "No favorite color found in state"

In [18]:

agent=create_agent(
    model=init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=0),
    tools=[update_favourite_color,read_favorite_color],
    checkpointer=InMemorySaver(),
    state_schema=CustomState

)

In [21]:
config={"configurable":{"thread_id":"5"}}
response=agent.invoke({"messages":[HumanMessage("What is my favorite color?")]}, config=config)

In [22]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is my favorite color?
================================== Ai Message ==================================
Tool Calls:
  read_favorite_color (0dc979d8-53ca-4fa3-998d-71e6e1881c60)
 Call ID: 0dc979d8-53ca-4fa3-998d-71e6e1881c60
  Args:
================================= Tool Message =================================
Name: read_favorite_color

No favorite color found in state
================================== Ai Message ==================================

[{'type': 'text', 'text': "I don't know your favorite color.", 'extras': {'signature': 'CqwCAXLI2nzuAVrn/CdSYQtR82BnMQDAeW6aIZKITIlCfTbXNsY9oTX7e42WnZBaah3B/y80y7NSn4Cmba1fmjW0+u7ZWsWaTvH7W6NeHuC0ibViog6lw0PLK72E6rRaa9gnPFbmaBXSmNRhucmhuQoIxiM+rr2ybDE2KPslOJEE0924/kI16tqMVwhHNwbgT2EwZfXZnTKp8Gbh/7/Im0840XDT1xBV4OOKEKiiwxT3wA+bP3FKpI+G38jPqYYpwpbzh28fStwJedvdKXXck1wLQ6cP8pulpqiZ0HbBitAR240DQ11YqgiD/Kj/K6uooPLD6Cxr5NB5Tt0ezKii5NABqANZnTUk8vSiP+brMhSxqzCmXKj

In [23]:
response=agent.invoke({"messages":[HumanMessage("ok, my favorite color is lemon yellow?")]}, config=config)

In [24]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is my favorite color?
================================== Ai Message ==================================
Tool Calls:
  read_favorite_color (0dc979d8-53ca-4fa3-998d-71e6e1881c60)
 Call ID: 0dc979d8-53ca-4fa3-998d-71e6e1881c60
  Args:
================================= Tool Message =================================
Name: read_favorite_color

No favorite color found in state
================================== Ai Message ==================================

[{'type': 'text', 'text': "I don't know your favorite color.", 'extras': {'signature': 'CqwCAXLI2nzuAVrn/CdSYQtR82BnMQDAeW6aIZKITIlCfTbXNsY9oTX7e42WnZBaah3B/y80y7NSn4Cmba1fmjW0+u7ZWsWaTvH7W6NeHuC0ibViog6lw0PLK72E6rRaa9gnPFbmaBXSmNRhucmhuQoIxiM+rr2ybDE2KPslOJEE0924/kI16tqMVwhHNwbgT2EwZfXZnTKp8Gbh/7/Im0840XDT1xBV4OOKEKiiwxT3wA+bP3FKpI+G38jPqYYpwpbzh28fStwJedvdKXXck1wLQ6cP8pulpqiZ0HbBitAR240DQ11YqgiD/Kj/K6uooPLD6Cxr5NB5Tt0ezKii5NABqANZnTUk8vSiP+brMhSxqzCmXKj

In [25]:
response=agent.invoke({"messages":[HumanMessage("change my favorite color to black?")]}, config=config)

In [26]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is my favorite color?
================================== Ai Message ==================================
Tool Calls:
  read_favorite_color (0dc979d8-53ca-4fa3-998d-71e6e1881c60)
 Call ID: 0dc979d8-53ca-4fa3-998d-71e6e1881c60
  Args:
================================= Tool Message =================================
Name: read_favorite_color

No favorite color found in state
================================== Ai Message ==================================

[{'type': 'text', 'text': "I don't know your favorite color.", 'extras': {'signature': 'CqwCAXLI2nzuAVrn/CdSYQtR82BnMQDAeW6aIZKITIlCfTbXNsY9oTX7e42WnZBaah3B/y80y7NSn4Cmba1fmjW0+u7ZWsWaTvH7W6NeHuC0ibViog6lw0PLK72E6rRaa9gnPFbmaBXSmNRhucmhuQoIxiM+rr2ybDE2KPslOJEE0924/kI16tqMVwhHNwbgT2EwZfXZnTKp8Gbh/7/Im0840XDT1xBV4OOKEKiiwxT3wA+bP3FKpI+G38jPqYYpwpbzh28fStwJedvdKXXck1wLQ6cP8pulpqiZ0HbBitAR240DQ11YqgiD/Kj/K6uooPLD6Cxr5NB5Tt0ezKii5NABqANZnTUk8vSiP+brMhSxqzCmXKj

In [27]:
response=agent.invoke({"messages":[HumanMessage("what is my favorite color after so many updates ?")]}, config=config)

In [28]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is my favorite color?
================================== Ai Message ==================================
Tool Calls:
  read_favorite_color (0dc979d8-53ca-4fa3-998d-71e6e1881c60)
 Call ID: 0dc979d8-53ca-4fa3-998d-71e6e1881c60
  Args:
================================= Tool Message =================================
Name: read_favorite_color

No favorite color found in state
================================== Ai Message ==================================

[{'type': 'text', 'text': "I don't know your favorite color.", 'extras': {'signature': 'CqwCAXLI2nzuAVrn/CdSYQtR82BnMQDAeW6aIZKITIlCfTbXNsY9oTX7e42WnZBaah3B/y80y7NSn4Cmba1fmjW0+u7ZWsWaTvH7W6NeHuC0ibViog6lw0PLK72E6rRaa9gnPFbmaBXSmNRhucmhuQoIxiM+rr2ybDE2KPslOJEE0924/kI16tqMVwhHNwbgT2EwZfXZnTKp8Gbh/7/Im0840XDT1xBV4OOKEKiiwxT3wA+bP3FKpI+G38jPqYYpwpbzh28fStwJedvdKXXck1wLQ6cP8pulpqiZ0HbBitAR240DQ11YqgiD/Kj/K6uooPLD6Cxr5NB5Tt0ezKii5NABqANZnTUk8vSiP+brMhSxqzCmXKj